In [ ]:
!pip install transformers

!pip install datasets

!pip install scikit-learn

!pip install torch

!pip install torch --extra-index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.optim import AdamW

os.environ['WANDB_DISABLED'] = 'true'

tokenizer = AutoTokenizer.from_pretrained("ltrctelugu/ltrc-distilbert")
model = AutoModelForSequenceClassification.from_pretrained("ltrctelugu/ltrc-distilbert", num_labels=3)

dataset = load_dataset("mounikaiiith/Telugu_Sentiment")

def encode_labels(example):
    label_mapping = {'neg': 0, 'neutral': 1, 'pos': 2}
    example['label'] = label_mapping[example['Sentiment']]
    return example

encoded_dataset = dataset.map(encode_labels)

def tokenize_function(example):
    return tokenizer(example['Sentence'], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = encoded_dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_dataset['train']
val_dataset = tokenized_dataset['validation']
test_dataset = tokenized_dataset['test']

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    adam_epsilon=1e-8
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(AdamW(model.parameters(), lr=3e-5, eps=1e-8), None)  # Specify AdamW optimizer
)

trainer.train()

train_results = trainer.evaluate(train_dataset)
print("Training set evaluation:", train_results)

val_results = trainer.evaluate(val_dataset)
print("Validation set evaluation:", val_results)

test_results = trainer.evaluate(test_dataset)
print("Test set evaluation:", test_results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Map:   0%|          | 0/7033 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.007000,1.010249,0.534473,0.372325,0.285661,0.534473
2,0.997200,1.006751,0.534473,0.372325,0.285661,0.534473
3,0.997700,1.005238,0.534473,0.372325,0.285661,0.534473
4,1.010600,1.003881,0.534473,0.372325,0.285661,0.534473
5,0.978800,1.005517,0.534473,0.372325,0.285661,0.534473
6,1.000100,1.009363,0.537037,0.380041,0.690261,0.537037
7,0.989700,1.009695,0.536182,0.378146,0.418089,0.536182
8,0.975600,1.008583,0.531339,0.392161,0.480182,0.531339
9,0.951700,1.018550,0.529630,0.388335,0.469380,0.529630
10,0.977800,1.019960,0.527635,0.390653,0.455583,0.527635


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Training set evaluation: {'eval_loss': 0.9561485648155212, 'eval_accuracy': 0.5536810439448758, 'eval_f1': 0.4299373568971321, 'eval_precision': 0.5930188623874287, 'eval_recall': 0.5536810439448758, 'eval_runtime': 88.3929, 'eval_samples_per_second': 278.292, 'eval_steps_per_second': 8.7, 'epoch': 10.0}
Validation set evaluation: {'eval_loss': 1.0199601650238037, 'eval_accuracy': 0.5276353276353276, 'eval_f1': 0.39065257156243016, 'eval_precision': 0.455582945034162, 'eval_recall': 0.5276353276353276, 'eval_runtime': 12.6257, 'eval_samples_per_second': 278.004, 'eval_steps_per_second': 8.712, 'epoch': 10.0}
Test set evaluation: {'eval_loss': 1.0003716945648193, 'eval_accuracy': 0.5400255936300299, 'eval_f1': 0.40502160226967165, 'eval_precision': 0.46681779512530314, 'eval_recall': 0.5400255936300299, 'eval_runtime': 25.3194, 'eval_samples_per_second': 277.771, 'eval_steps_per_second': 8.689, 'epoch': 10.0}
